# Calculos Necessarios

## 1. Importacao/Limpeza/Preparacao

In [1]:
import pandas as pd
import numpy as np

# Só pra facilitar prints legíveis no notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)


In [2]:
csv_path = "/Users/christophe/Downloads/history.csv" 

# Leitura garantindo que a coluna 'date' vire datetime
df = pd.read_csv(csv_path, parse_dates=['date'], infer_datetime_format=True)

# Normaliza nomes de colunas para lowercase (evita erros de capitalização)
df.columns = [c.strip().lower() for c in df.columns]


print("Colunas do CSV:", df.columns.tolist())
print("\nTipos de dados:")
print(df.dtypes)
print("\nPrimeiras linhas:")
display(df.head())


/var/folders/lh/kgkxn3f536nfn07l9g_kbxb00000gn/T/ipykernel_2179/2451931521.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(csv_path, parse_dates=['date'], infer_datetime_format=True)


Colunas do CSV: ['id', 'ticker_name', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'stock_splits', 'date', 'created_at']

Tipos de dados:
id                       int64
ticker_name             object
open                   float64
high                   float64
low                    float64
close                  float64
volume                   int64
dividends                int64
stock_splits             int64
date            datetime64[ns]
created_at              object
dtype: object

Primeiras linhas:


,id,ticker_name,open,high,low,close,volume,dividends,stock_splits,date,created_at
0,1,AAPL,231.779999,232.729996,231.339996,232.350006,5111585,0,0,2025-08-18 09:30:00,2025-09-17 13:55:24.019793+00
1,2,AAPL,232.350006,233.119995,230.610001,230.660004,4236945,0,0,2025-08-18 10:00:00,2025-09-17 13:55:24.019793+00
2,3,AAPL,230.619995,231.149994,230.369995,230.509995,2764838,0,0,2025-08-18 10:30:00,2025-09-17 13:55:24.019793+00
3,4,AAPL,230.530502,231.220001,230.470001,230.860001,1934703,0,0,2025-08-18 11:00:00,2025-09-17 13:55:24.019793+00
4,5,AAPL,230.865005,231.039993,230.500000,230.904999,1669583,0,0,2025-08-18 11:30:00,2025-09-17 13:55:24.019793+00


In [3]:
expected_cols = ['ticker_name', 'open', 'high', 'low', 'close', 'volume', 'date']

missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Faltando colunas esperadas no CSV: {missing}. Verifique nomes/maiúsculas.")

# Garantir numericidade nas colunas de preço/volume
numeric_cols = ['open', 'high', 'low', 'close', 'volume']
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')  # converte, coloca NaN se não for numérico

# Verificação de valores nulos nas colunas numéricas e data
print("\nNulos por coluna:")
print(df[numeric_cols + ['date']].isnull().sum())

df = df.dropna(subset=['date'])

print("\nTamanho do dataframe após drop de datas nulas:", len(df))



Nulos por coluna:
open      0
high      0
low       0
close     0
volume    0
date      0
dtype: int64

Tamanho do dataframe após drop de datas nulas: 105


In [4]:
print("Tickers únicos no arquivo:", df['ticker_name'].unique())


Tickers únicos no arquivo: ['AAPL']


## 2. Calculos separados

### 2.1. Media

In [5]:
def calcular_media(df, ticker, coluna, periodo):
    colunas_validas = ['open', 'high', 'low', 'close', 'volume']
    periodos_validos = ['day', 'week', 'month', 'year']

    if coluna not in colunas_validas:
        raise ValueError(f"Coluna inválida! Escolha entre: {colunas_validas}")
    if periodo not in periodos_validos:
        raise ValueError(f"Período inválido! Escolha entre: {periodos_validos}")

    # Filtrar o ticker
    df_ticker = df[df['ticker_name'] == ticker].copy()
    df_ticker['date'] = pd.to_datetime(df_ticker['date'])
    ultima_data = df_ticker['date'].max()

    # Determinar o intervalo
    if periodo == 'day':
        data_inicial = ultima_data - pd.Timedelta(days=1)
    elif periodo == 'week':
        data_inicial = ultima_data - pd.Timedelta(weeks=1)
    elif periodo == 'month':
        data_inicial = ultima_data - pd.DateOffset(months=1)
    else:  # year
        data_inicial = ultima_data - pd.DateOffset(years=1)

    df_periodo = df_ticker[df_ticker['date'] >= data_inicial]

    return round(df_periodo[coluna].mean(), 2) #Calculo da media


### 2.2. Variacao percentual

In [6]:
def calcular_variacao(df, ticker, coluna, periodo):
    
    colunas_validas = ['open', 'high', 'low', 'close', 'volume']
    periodos_validos = ['day', 'week', 'month', 'year']

    if coluna not in colunas_validas:
        raise ValueError(f"Coluna inválida! Escolha entre: {colunas_validas}")
    if periodo not in periodos_validos:
        raise ValueError(f"Período inválido! Escolha entre: {periodos_validos}")

    df_ticker = df[df['ticker_name'] == ticker].copy()
    df_ticker['date'] = pd.to_datetime(df_ticker['date'])
    ultima_data = df_ticker['date'].max()

    if periodo == 'day':
        data_inicial = ultima_data - pd.Timedelta(days=1)
    elif periodo == 'week':
        data_inicial = ultima_data - pd.Timedelta(weeks=1)
    elif periodo == 'month':
        data_inicial = ultima_data - pd.DateOffset(months=1)
    else:
        data_inicial = ultima_data - pd.DateOffset(years=1)

    df_periodo = df_ticker[df_ticker['date'] >= data_inicial].sort_values('date')

    if df_periodo.empty:
        return None

    primeiro = df_periodo[coluna].iloc[0]
    ultimo = df_periodo[coluna].iloc[-1]

    return round(((ultimo - primeiro) / primeiro) * 100, 2) if primeiro != 0 else None #Calculo da variacao


### 2.3. Volatilidade Simples

In [7]:
def calcular_volatilidade(df, ticker, coluna, periodo):
   
    colunas_validas = ['open', 'high', 'low', 'close', 'volume']
    periodos_validos = ['day', 'week', 'month', 'year']

    if coluna not in colunas_validas:
        raise ValueError(f"Coluna inválida! Escolha entre: {colunas_validas}")
    if periodo not in periodos_validos:
        raise ValueError(f"Período inválido! Escolha entre: {periodos_validos}")

    df_ticker = df[df['ticker_name'] == ticker].copy()
    df_ticker['date'] = pd.to_datetime(df_ticker['date'])
    ultima_data = df_ticker['date'].max()

    if periodo == 'day':
        data_inicial = ultima_data - pd.Timedelta(days=1)
    elif periodo == 'week':
        data_inicial = ultima_data - pd.Timedelta(weeks=1)
    elif periodo == 'month':
        data_inicial = ultima_data - pd.DateOffset(months=1)
    else:
        data_inicial = ultima_data - pd.DateOffset(years=1)

    df_periodo = df_ticker[df_ticker['date'] >= data_inicial]

    return round(df_periodo[coluna].std(), 2) #Calculo da volatilidade


In [8]:
ticker = 'AAPL'
coluna = 'close'
periodo = 'month'

print(f"Média ({coluna}):", calcular_media(df, ticker, coluna, periodo))
print(f"Variação percentual ({coluna}):", calcular_variacao(df, ticker, coluna, periodo))
print(f"Volatilidade ({coluna}):", calcular_volatilidade(df, ticker, coluna, periodo))


Média (close): 232.13
Variação percentual (close): 2.71
Volatilidade (close): 4.58


## 3. Mesmo calculo para todas as colunas

In [9]:
def analisar_todas_colunas(df, ticker, periodo):
    # 🔹 Filtra o ticker
    df_ticker = df[df['ticker_name'] == ticker].copy()
    df_ticker['date'] = pd.to_datetime(df_ticker['date'])
    
    ultima_data = df_ticker['date'].max()

    if periodo == 'day':
        data_inicial = ultima_data - pd.Timedelta(days=1)
    elif periodo == 'week':
        data_inicial = ultima_data - pd.Timedelta(weeks=1)
    elif periodo == 'month':
        data_inicial = ultima_data - pd.DateOffset(months=1)
    elif periodo == 'year':
        data_inicial = ultima_data - pd.DateOffset(years=1)
    else:
        raise ValueError("Período inválido! Escolha entre: 'day', 'week', 'month', 'year'.")

    # 🔹 Filtra o período
    df_periodo = df_ticker[df_ticker['date'] >= data_inicial]

 
    colunas = ['open', 'high', 'low', 'close', 'volume']

    print(f"\n📊 Resultados para {ticker} — Período: {periodo}\n")

    # 🔹 Calcula para cada coluna
    for coluna in colunas:
        media = round(df_periodo[coluna].mean(), 2)
        primeiro = df_periodo[coluna].iloc[0]
        ultimo = df_periodo[coluna].iloc[-1]
        variacao = round(((ultimo - primeiro) / primeiro) * 100, 2) if primeiro != 0 else np.nan
        volatilidade = round(df_periodo[coluna].std(), 2)

        print(f"{coluna}:")
        print(f"  Média: {media}")
        print(f"  Variação %: {variacao}")
        print(f"  Volatilidade: {volatilidade}\n")


In [10]:
analisar_todas_colunas(df, ticker='AAPL', periodo='month')



📊 Resultados para AAPL — Período: month

open:
  Média: 232.17
  Variação %: 3.37
  Volatilidade: 4.61

high:
  Média: 232.81
  Variação %: 2.94
  Volatilidade: 4.63

low:
  Média: 231.46
  Variação %: 3.08
  Volatilidade: 4.57

close:
  Média: 232.13
  Variação %: 2.71
  Volatilidade: 4.58

volume:
  Média: 4020693.06
  Variação %: -51.95
  Volatilidade: 2761607.21



## 4. Calculo de RSI

### 4.1. Oque é o RSI e pra que serve


**O que é o RSI?**  
O RSI (Relative Strength Index, ou Índice de Força Relativa) é um indicador que mede a **força** dos movimentos de preço de um ativo. Ele compara os ganhos recentes com as perdas recentes e apresenta um valor entre **0 e 100**.

**Como interpretar (de forma simples):**
- **RSI > 70** → ativo possivelmente **sobrecomprado** (subiu demais; atenção para correção).  
- **RSI entre 30 e 70** → zona **neutra** (movimento considerado “normal”).  
- **RSI < 30** → ativo possivelmente **sobrevendido** (caiu demais; pode haver recuperação).

**O que ele serve:**  
- Detectar extremos de mercado (momentos em que o preço pode corrigir).  
- Ajudar a escolher pontos de entrada/saída (quando combinado com outras análises).  
- Avaliar se um movimento de preço tem **momentum** (força) ou se está perdendo força.

**Exemplo prático:**  
Se o RSI chega a valores muito altos (80–90) enquanto o preço ainda sobe, isso pode indicar *euforia* e maior chance de correção; se o RSI cai enquanto o preço ainda está alto, isso pode indicar *divergência* (sinal de fraqueza).


In [33]:
def calcular_RSI_14dias(df, ticker):
   
    df_ticker = df[df['ticker_name'] == ticker].copy()
    df_ticker['date'] = pd.to_datetime(df_ticker['date'])
    
    df_diario = df_ticker.groupby(df_ticker['date'].dt.date)['close'].last().reset_index()
    df_diario.rename(columns={'date':'dia'}, inplace=True)
    
    df_diario = df_diario.tail(14+1).copy()  # +1 para poder calcular delta do primeiro dia
    
    df_diario['delta'] = df_diario['close'].diff()
    
    df_diario['ganho'] = df_diario['delta'].apply(lambda x: x if x > 0 else 0)
    df_diario['perda'] = df_diario['delta'].apply(lambda x: -x if x < 0 else 0)
    
    media_ganhos = df_diario['ganho'].iloc[1:15].mean() 
    media_perdas = df_diario['perda'].iloc[1:15].mean()
    
    RSI_list = [np.nan]  # primeiro dia não tem RSI
    
    # Calcula RSI para cada dia
    for i in range(1, len(df_diario)):
        ganho = df_diario['ganho'].iloc[i]
        perda = df_diario['perda'].iloc[i]
        
        # Atualiza médias usando suavização exponencial
        media_ganhos = (media_ganhos * 13 + ganho) / 14
        media_perdas = (media_perdas * 13 + perda) / 14
        
        RS = media_ganhos / media_perdas if media_perdas != 0 else np.inf
        RSI = 100 - (100 / (1 + RS))
        RSI_list.append(RSI)
    
    df_diario['RSI'] = RSI_list
    
    df_result = df_diario.tail(14)
    
    print(f"\n📈 RSI — {ticker} — Últimos 14 dias")
    print(df_result[['dia', 'close', 'RSI']])
    
    return df_result[['dia', 'close', 'RSI']]


In [34]:
df_rsi = calcular_RSI_14dias(df, 'AAPL')


📈 RSI — AAPL — Últimos 14 dias
           dia       close        RSI
7   2025-08-27  229.914993  66.201296
8   2025-08-28  231.554993  67.499614
9   2025-08-29  231.773193  67.677517
10  2025-09-02  227.415405  60.549047
11  2025-09-03  235.615005  67.488170
12  2025-09-04  237.410004  68.782660
13  2025-09-05  238.994598  69.921222
14  2025-09-08  238.279999  68.704269
15  2025-09-09  235.679993  64.318017
16  2025-09-10  227.259995  52.605211
17  2025-09-11  228.410004  53.841594
18  2025-09-12  233.970001  59.361349
19  2025-09-15  237.065002  62.079736
20  2025-09-16  238.654999  63.432996


**Análise do RSI (AAPL):**

- O RSI da Apple (AAPL) ficou entre 52,6 e 69,9, mostrando força compradora moderada, mas sem atingir a zona de sobrecompra (acima de 70).

- Nos primeiros dias, o RSI subiu gradualmente até quase 70, indicando tendência de alta consistente. Depois, houve uma leve correção com queda para a faixa dos 50, seguida por recuperação gradual até os 60.

👉 Resumo:
Tendência: leve alta, sem excesso.
Sem sinais de sobrecompra ou sobrevenda.
Mercado equilibrado, com força compradora estável.